In [1]:
import pandas as pd
import numpy as np
import tensorflow.keras as K
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import pickle
from tqdm import tqdm
#from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Model, Sequential
#print(tf.config.list_physical_devices('GPU'))

In [2]:
base_model = ResNet101(include_top = False, weights='imagenet',input_shape=(224,224,3), pooling = 'avg')
model = Model(base_model.input, base_model.layers[-4].output)

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [4]:

data = pd.read_csv('../../IU_data/reports_data/all_data.csv')
images_path='/media/zaheer/Data/Image_Text_Datasets/IU_Xray/Image-Text/IIU/NLMCXR_png/'
#images_path='/media/zaheer/Data/Image_Text_Datasets/IU_Xray/Image-Text/IIU/train2014_resized/'

data.columns

Index(['Image Index', 'Patient ID', 'Findings', 'Impression', 'Caption',
       'Manual Tags', 'Reports', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')

In [5]:

report_features = {}
image_features = {}
for images in data['Image Index']:
    image_path=images_path+images
    img = image.load_img(image_path, target_size=(224, 224,3))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    #x=x/255.0
    # Add one more dimension
#         x = np.expand_dims(x, axis=0)

    x=preprocess_input(x)
    x = np.reshape(x,(-1,224,224,3))
    feature = model.predict(x)
    
    r_id = images.split('_')[0]
    
    #print(r_id)
    if (r_id) not in report_features:
        report_features[r_id] = []
        report_features[r_id].append(feature)
    else:
        report_features[r_id].append(feature)
    
    image_features[images] = feature

In [6]:
path='/media/zaheer/Data/Image_Text_Datasets/IU_Xray/latest/One_Image_norm_abnorm_split/r2gen_annotations/'

In [7]:
with open(path+'resnet_report_features.pkl','wb') as file:
    pickle.dump(report_features, file, protocol=2)

In [8]:
with open(path+'resnet_image_features.pkl','wb') as file:
    pickle.dump(image_features, file, protocol=2)